In [ ]:
/storage/quangtm/KIE_invoice_minimal/receipt_1488_rs.jpg
/storage/quangtm/KIE_invoice_minimal/receipt_1488_rs.json

In [13]:
def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Tính diện tích phần giao nhau
    if (xB - xA + 1) <0:
        return 0
    if ((yB - yA + 1))<0:
        return 0
    interArea = (xB - xA + 1) * (yB - yA + 1)

    # Diện tích của predicted và ground-truth bounding box
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Tính IoU = diện tích phần giao nhau chia diện tích phần tổng hợp
    # Diện tích phần hợp = tổng diện tích trừ diện tích phần giao
    iou = interArea / float(boxAArea)
    iou123= interArea / float(boxBArea)
    if iou==0:
        iou=0
    else:
        iou=iou+iou123
    # Trả về giá trị iou
    return iou

In [14]:
import json
import os
lisdir=os.listdir("/storage/quangtm/KIE_invoice_minimal/json3_line/")
from PIL import ImageDraw, ImageFont,Image


from backend.models import load_text_detect, load_text_recognize, load_saliency
detector = load_text_recognize()
import cv2
import numpy as np
import re

text_detector = load_text_detect()
def no_accent_vietnamese(text):
    text1=[]
    for s in text:
        s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
        s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
        s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
        s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
        s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
        s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
        s = re.sub(r'[ìíịỉĩ]', 'i', s)
        s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
        s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
        s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
        s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
        s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
        s = re.sub(r'[Đ]', 'D', s)
        s = re.sub(r'[đ]', 'd', s)
        text1.append(s)
    return text1


def no_accent_vietnamese_token(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s


from backend.backend_utils import (
    NpEncoder,
    run_ocr,
    make_warp_img,
    resize_and_pad,
    get_group_text_line,
)
from backend.text_detect.text_detector import get_textbox  
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype="float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect
from backend.text_detect.config import craft_config
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array(
        [[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]],
        dtype="float32",
    )

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped, M
import copy
color=(255,0,0)
regex = r"^\,\d"
regex1=r"([0-9]{1})"
regex2 = r"^\.\d"
import re
font = ImageFont.load_default()
for ko in range(len(lisdir)):
    
    box_list=[]
    list_coor=[]
    if "json" in lisdir[ko]:
        path_json="/storage/quangtm/KIE_invoice_minimal/json3_line/"+lisdir[ko]
        path_txt="/storage/quangtm/KIE_invoice_minimal/data_txt/"+lisdir[ko][:-4]+"txt"
#         path_json="/storage/quangtm/KIE_invoice_minimal/receipt_1488_rs.json"
#         print(path_json)
        path_img="/storage/quangtm/KIE_invoice_minimal/json3_line/"+lisdir[ko][:-4]+"jpg"
#         path_json="/storage/quangtm/KIE_invoice_minimal/receipt_1488_rs.json"
#         print(path_json)
#         path_img="/storage/quangtm/KIE_invoice_minimal/receipt_1488_rs.jpg"
        img=cv2.imread(path_img)

        image=Image.open(path_img)
        draw = ImageDraw.Draw(image, "RGBA")
        heatmap, textboxes = get_textbox(
        text_detector,
        img,
        craft_config["canvas_size"],
        craft_config["mag_ratio"],
        craft_config["text_threshold"],
        craft_config["link_threshold"],
        craft_config["low_text"],
        False,
        "cpu",
        )
        warped_img=img
        cells, heatmap, textboxes = run_ocr(
        text_detector, detector, img, craft_config
        )
        _, lines = get_group_text_line(heatmap, textboxes)

        for line_id, cell in zip(lines, cells):
            cell["group_id"] = line_id
        img_info = dict()
        sub_h_img, sub_w_img, _ = warped_img.shape
        img_info["h_origin"] = sub_h_img
        img_info["w_origin"] = sub_w_img
        img_info["cells"] = cells

        group_ids = np.array([i for i in lines])
        merged_cells = []
        thickness = 2
        Box_sort_list_new,line_arr=[],[]
        merge_list=[]
        listtext=[]
        list_boxesa=[]
        for group_id in range(max(group_ids) + 1):
            indexes = np.where(group_ids == group_id)[0]

            if len(indexes) == 0:
                continue
            filter_cells = [cells[index] for index in indexes]
            filter_cells = sorted(filter_cells, key=lambda x: np.mean(x["poly"][::2]) )
            so_list=[]
            
            text1=filter_cells[0]['vietocr_text']
            result = re.match(regex, text1)
            if result:
                result1a=True
            else:
                result = re.match(regex1, text1)
                if result:
                    result1a=True
                else:
                    result = re.match(regex2, text1)
                    if result:
                        result1a=True
                    else:
                        result1a=False

            for i in range(1,len(filter_cells),1):
                text=filter_cells[i]['vietocr_text']
        #         print(text)
                result = re.match(regex, text)
                if result:
                    result1=True
                else:
                    result = re.match(regex1, text)
                    if result:
                        result1=True
                    else:
                        result = re.match(regex2, text)
                        if result:
                            result1=True
                        else:
                            result1=False


                if (result1a==True) and (result1==True):
        #             so_list.append(text)
                    so_list.append(i-1) 
                    so_list.append(i) 
                result1a=result1
            so_list=list(set(so_list))
            check=False
            boroi=False
            string_list=""
            id_text=-1
            for i in range(len(filter_cells)):
                id_text=id_text+1
                x1b,y1b,x2b,y2b,x3b,y3b,x4b,y4b=filter_cells[i-1]['poly']
                textn=filter_cells[i-1]['vietocr_text']
                group=filter_cells[i-1]['group_id']
                x1,y1,x2,y2,x3,y3,x4,y4=filter_cells[i]['poly']
                text=filter_cells[i]['vietocr_text']
                group=filter_cells[i]['group_id']
                if i not in so_list:
                    if check==True :
                        listtext.append(string_list)

                        list_boxesa.append([left_pont,top_point,x3b,y3b])
                        new_poly=[left_pont,top_point,x2b,y2b,x3b,y3b,right_point,bottom_point]
                        tmp_cell = copy.deepcopy(filter_cells[0])
                        tmp_cell["poly"] = new_poly
                        tmp_cell["vietocr_text"] = string_list
                        tmp_cell["group_id"]=filter_cells[0]["group_id"]+1000000
                        merged_cells.append(copy.deepcopy(tmp_cell))
                        string_list=""
                    merged_cells.append(filter_cells[i])
                    listtext.append(text)
                    list_boxesa.append([x1,y1,x3,y3])
                    check=False
                else:
        #             print(text)
                    if (abs(x2b-x1)>min(abs(x2-x1),abs(x2b-x1b)) ) and (check==True):
                        listtext.append(string_list)
                        list_boxesa.append([left_pont,top_point,x3b,y3b])
                        new_poly=[left_pont,top_point,x2b,y2b,x3b,y3b,right_point,bottom_point]
                        tmp_cell = copy.deepcopy(filter_cells[0])
                        tmp_cell["poly"] = new_poly
                        tmp_cell["vietocr_text"] = string_list
                        tmp_cell["group_id"]=filter_cells[0]["group_id"]+1000000
                        merged_cells.append(copy.deepcopy(tmp_cell))

                        string_list=""
                        left_pont,top_point,right_point,bottom_point=x1,y1,x4,y4
                    if check==False:
        #                 print("text123",text)
                        left_pont,top_point,right_point,bottom_point=x1,y1,x4,y4
                    if i == len(filter_cells)-1:
                        string_list=string_list+text
        #                 print("string_lists",string_list)

                        listtext.append(string_list)

                        list_boxesa.append([left_pont,top_point,x3,y3])
                        new_poly=[left_pont,top_point,x2,y2,x3,y3,right_point,bottom_point]
                        tmp_cell = copy.deepcopy(filter_cells[0])
                        tmp_cell["poly"] = new_poly
                        tmp_cell["group_id"]=filter_cells[0]["group_id"]+1000000
                        tmp_cell["vietocr_text"] = string_list
                        merged_cells.append(copy.deepcopy(tmp_cell))

                    string_list=string_list+text
        #             print("string_list_fil",string_list,"i",i,len(filter_cells))
                    check=True   
        bbox=list_boxesa
        text=listtext

        text=no_accent_vietnamese(text)
        # path_img="/media/quang/New Volume/DATAOCR/json3_line/"+lisdir[i][:-4]+"jpg"
        with open(path_json) as json_file:
            data=json.load(json_file)
        list_box_json=[]
        list_label_json=[]
        for i in data["shapes"]:
        #     print(i["label"])
        #     print("i[""]",len(i["points"]))
            try:
                if len(i["points"])==4:
                    point1,point2,point3,point4=i["points"]
                else:
                    point1,point3=i["points"]
                    point2=[point3[0],point1[1]]
                    point4=[point1[0],point3[1]]
            except:
                continue
                
            box = [point1[0],point1[1],point2[0],point2[1], point3[0], point3[1],point4[0],point4[1]]
            left_x= int(min(point1[0],point2[0],point3[0],point4[0] ) )
            top_y= int(min(point1[1],point2[1],point3[1],point4[1]))
            right_x= int(max(point2[0],point3[0] ,point1[0],point2[0]))
            bot_y=int(max(point3[1],point4[1],point1[1],point2[1]))
            box=[left_x,top_y,right_x,bot_y]
            list_box_json.append(box)
            list_label_json.append(i["label"])
        image=warped_img
        image=cv2.imread(path_img)
        img = resize_and_pad(img, size=1024, pad=False)
        image = Image.fromarray(image)
        draw = ImageDraw.Draw(image, "RGBA")
        list_label=[]
        list_box=[]
        list_box_dacheck=[]
        list_text=[]
        list_array=[x[:] for x in [[]] * len(merged_cells)] 
        list_array_ignore=[]
        for i in range(len(merged_cells)):
            boxes=merged_cells[i]['poly']
            text=merged_cells[i]['vietocr_text']
            box=boxes[0],boxes[1],boxes[2],boxes[3],boxes[4],boxes[5],boxes[6],boxes[7]
            left_x= int(min(box[0],box[2],box[4],box[6]))
            top_y= int(min(box[1],box[5],box[7],box[3]))
            right_x= int(max(box[2],box[0],box[6],box[4]))
            bot_y=int(max(box[5],box[7],box[1],box[3]))
            box=[left_x,top_y,right_x,bot_y]
            max_iou=0
            max_id=-1
            max_id_new=-1
            for j in range(len(list_box_json)):
                box_ground=list_box_json[j]
                st=bb_intersection_over_union(box,box_ground)
                if st > max_iou:
                    max_id_new=i
                    max_id=j
                    max_iou=st
                    list_box_dacheck.append(max_id)
        #     print(max_id_new,max_id)
            if max_id==-1:
                list_array_ignore.append(i)
            else:
                list_array[max_id].append(max_id_new)
            if max_id==-1:
                label_draw="other"
            else:
                label_draw=str(list_label_json[max_id])
        for i in range(len(list_label_json)):
            for j in list_array[i]:
                boxes=merged_cells[j]['poly']
                text=merged_cells[j]['vietocr_text']
                box=boxes[0],boxes[1],boxes[2],boxes[3],boxes[4],boxes[5],boxes[6],boxes[7]
                left_x= int(min(box[0],box[2],box[4],box[6]))
                top_y= int(min(box[1],box[5],box[7],box[3]))
                right_x= int(max(box[2],box[0],box[6],box[4]))
                bot_y=int(max(box[5],box[7],box[1],box[3]))
                box=[left_x,top_y,right_x,bot_y]
                label_draw=list_label_json[i]
                list_box.append(box)
                label_draw=label_draw.replace("othẻ","other")
                if label_draw!="other":
                    if len(list_array[i])==1:
                        label_draw="s-"+label_draw
                    elif j==list_array[i][0]:
                        label_draw="b-"+label_draw
                    elif j==list_array[i][-1] :
                        label_draw="e-"+label_draw
                    else:
                        label_draw="i-"+label_draw
                list_label.append(label_draw)
                text=no_accent_vietnamese_token(text)
                list_text.append(text)
        for i in list_array_ignore:
            boxes=merged_cells[i]['poly']
            text=merged_cells[i]['vietocr_text']
            box=boxes[0],boxes[1],boxes[2],boxes[3],boxes[4],boxes[5],boxes[6],boxes[7]
            left_x= int(min(box[0],box[2],box[4],box[6]))
            top_y= int(min(box[1],box[5],box[7],box[3]))
            right_x= int(max(box[2],box[0],box[6],box[4]))
            bot_y=int(max(box[5],box[7],box[1],box[3]))
            box=[left_x,top_y,right_x,bot_y]
            list_box.append(box)
            label_draw="other"
            label_draw=label_draw.replace("othẻ","other")
                   
            list_label.append(label_draw)
            text=no_accent_vietnamese_token(text)
            list_text.append(text)




#         list_box_con_lai=list(set(list_box_dacheck))
#         list_box_con_lai = np.array([i for i in list_box_con_lai])

            
            
        list_box_con_lai=list(set(list_box_dacheck))
        list_box_con_lai = np.array([i for i in list_box_con_lai])
        image=cv2.imread(path_img)
        for i in range(len(list_box_json)):
            indexes = np.where(list_box_con_lai == i)[0]
            if len(indexes)==0:
                list_label.append("s-"+str(list_label_json[i]))
                x1,y1,x3,y3=list_box_json[i]

                warped_img=image[list_box_json[i][1]:list_box_json[i][3],list_box_json[i][0]:list_box_json[i][2]]
        #         print(warped_img.shape)
                pil_img = Image.fromarray(warped_img)
                s = detector.predict(pil_img)
                text=no_accent_vietnamese_token(s)
                list_text.append(text)
                list_box.append(list_box_json[i])

        # image=cv2.imread(path_img)
        img = resize_and_pad(img, size=1024, pad=False)
        image = Image.fromarray(image)
        draw = ImageDraw.Draw(image, "RGBA")
        for i in range(len(list_box)):
            box=list_box[i]
            label_draw=list_label[i]
            text_draw=list_text[i]
            draw.rectangle(box, outline="red")
            draw.text((box[0]+10, box[1]), text=label_draw, fill="red", font=font)
            draw.text((box[0]+10, box[1]-10), text=text_draw, fill="blue", font=font)
        path_txt="/storage/quangtm/KIE_invoice_minimal/data_txt/"+lisdir[ko][:-4]+"txt"
        f=open(path_txt,"w+")
        j=0
        for i in range(len(list_box)):
            x1,y1,x3,y3=list_box[i]
            label_1=list_label[i]
            line_text=str(int(x1))+","+str(int(y1))+","+str(int(x3))+","+str(int(y1))+","+str(int(x3))+","+str(int(y3))+","+str(int(x1))+","+str(int(y3))+","+str(list_label[i])+","+str(list_text[i])
            j+=1
            f.write(line_text)
            f.write('\n')
    
        image.save("/storage/quangtm/KIE_invoice_minimal/imagedraw/"+str(i)+".jpg")
        image.show()
#         break



File exists: /home/quangtm/.cache/gdown/https-COLON--SLASH--SLASH-drive.google.com-SLASH-uc-QUESTION-id-EQUAL-1nTKlEog9YFK74kPyX0qLwCWi60_YHHk4
- Function run_ocr run in 1.57820129's
- Function get_group_text_line run in 0.16954923's


In [ ]:
import os
import json
import cv2
def convert_labelme(img_dir, anns_dir, json_anns_dir):
    dic = {'version': '4.5.6', 'flags': {}, 'imageData': None}
    for anns_name in os.listdir(anns_dir):
        anns_path = os.path.join(anns_dir, anns_name)
       
        # print(img_path)
        img_name = anns_name.replace('.txt', '.jpg')
        img_path = os.path.join(img_dir, img_name)
        
        dic['imagePath'] = img_name
        print(img_path)
        img = cv2.imread(img_path)
        h, w = img.shape[:2]
        dic['imageHeight'] = h
        dic['imageWeight'] = w
        list_shapes = []
        with open(anns_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                dic_shapes = {}
                line = line.strip().split(',')
                labe=line[8]
                line = [int(li) for li in line[:8]]
                points = [[line[0], line[1]], [line[2], line[3]], [line[4], line[5]], [line[6], line[7]]]
                dic_shapes['label'] = str(labe)
                dic_shapes['points'] = points
                dic_shapes['group_id'] = None
                dic_shapes['shape_type'] = 'polygon'
                dic_shapes['flags'] = {}
                list_shapes.append(dic_shapes)
        dic['shapes'] = list_shapes
        with open(os.path.join(json_anns_dir, img_name.replace('.jpg', '.json')), 'w') as out_file:
            json.dump(dic, out_file)

convert_labelme('/storage/quangtm/KIE_invoice_minimal/json3_line/','/storage/quangtm/KIE_invoice_minimal/data_txt/','/storage/quangtm/KIE_invoice_minimal/json3_line123/')